In [10]:
from pyhive import trino
from getpass import getpass
import pandas as pd
import awswrangler as wr
import seaborn as sns

In [3]:
conn = trino.connect(
    host = 'trino.de.in.devneon.com.br',
    port = 443,
    protocol = 'https',
    username = getpass('Insert your User u00xxxx: '),  
    password = getpass('Insert password: '),
)
print(conn)

Insert your User u00xxxx:  ········
Insert password:  ········


In [6]:
query_card_transactions = f"""
SELECT 
    distinct od.clientid,
    DAY(current_date - MAX(transactiondate)) as RECENCY,
    COUNT(transactiondate) as FREQUENCY,
    SUM(convertedvalue) as MONETARY
FROM data_processed.operacoes_diaria od
LEFT JOIN neondw_bi.dimension_client dc on od.clientid = dc.clientid
WHERE operationstatus = 'EFETIVADO' 
AND product = 'CARTAO CREDITO'
AND subproduct in ('FISICO CREDITO NACIONAL', 
					'VIRTUAL CREDITO NACIONAL', 
					'FISICO CREDITO INTERNACIONAL',
					'VIRTUAL CREDITO INTERNACIONAL')
AND transactiondate between date_add('day', -60, current_date) and current_date
AND persontype = 'PF'
GROUP BY 1
ORDER BY RECENCY asc
"""

In [7]:
df = pd.read_sql(query_card_transactions, conn)

In [9]:
wr.s3.to_parquet(df, 's3://neon-datascience-sandbox/segmentacao-cartoes/card-use')

{'paths': ['s3://neon-datascience-sandbox/segmentacao-cartoes/card-use'],
 'partitions_values': {}}

In [25]:
querie_dimension = f"""
SELECT
	clientid,
	presumedincome,
	age,
	ageneon,
    flaginviteduser
FROM hive.neondw_bi.dimension_client
WHERE clientstatus = 'Ativo'
AND persontype = 'PF'
"""

In [26]:
df_dimension = pd.read_sql(querie_dimension, conn)

In [27]:
wr.s3.to_parquet(df_dimension, 's3://neon-datascience-sandbox/segmentacao-cartoes/dimension')

{'paths': ['s3://neon-datascience-sandbox/segmentacao-cartoes/dimension'],
 'partitions_values': {}}

In [44]:
querie_gh = f"""
SELECT
    clientid,
    CASE
        WHEN score <= 1 THEN NULL
        WHEN score <= 33 THEN 10
        WHEN score <= 44 THEN 9
        WHEN score <= 60 THEN 8
        WHEN score <= 70 THEN 7
        WHEN score <= 82 THEN 6
        WHEN score <= 106 THEN 5
        WHEN score <= 125 THEN 4
        WHEN score <= 152 THEN 3
        WHEN score <= 202 THEN 2
    ELSE 1
    END AS GH_VL,
    MAX(reference_date) as date 
FROM hive.boavista.customizado_neon_v2 A
LEFT JOIN neondw_bi.dimension_client B ON A.document_nu = B.cpf_cnpj
WHERE persontype = 'PF'
AND clientstatus = 'Ativo'
GROUP BY 1,2
"""

In [45]:
df_gh = pd.read_sql(querie_gh, conn)

In [47]:
del df_gh['date']

In [48]:
wr.s3.to_parquet(df_gh, 's3://neon-datascience-sandbox/segmentacao-cartoes/gh')

{'paths': ['s3://neon-datascience-sandbox/segmentacao-cartoes/gh'],
 'partitions_values': {}}

In [49]:
querie_viracredito = f"""
SELECT 
	idcliente as clientid,
	1 as tem_viracredito
FROM refined.products.cards_crdt_card_viracredito_fact
WHERE flag in ('Concessão', 'Aumento de limite')
AND engagement_ccc_dt <= current_date
"""

In [50]:
df_viracredito = pd.read_sql(querie_viracredito, conn)

In [51]:
wr.s3.to_parquet(df_viracredito, 's3://neon-datascience-sandbox/segmentacao-cartoes/viracredito')

{'paths': ['s3://neon-datascience-sandbox/segmentacao-cartoes/viracredito'],
 'partitions_values': {}}

In [56]:
querie_account_balance = f"""
    select
        clientid,
        sum(totalbalancecumulatedcheckingaccount) ACCOUNT_BALANCE_VL
    from
        data_processed.accountbalance
    where
        data between date_add('day', -60, current_date) and current_date
    group by 1 """

In [57]:
df_account_balance = pd.read_sql(querie_account_balance, conn)

In [58]:
wr.s3.to_parquet(df_account_balance, 's3://neon-datascience-sandbox/segmentacao-cartoes/account-balance')

{'paths': ['s3://neon-datascience-sandbox/segmentacao-cartoes/account-balance'],
 'partitions_values': {}}

In [63]:
querie_limite_credito = f"""
SELECT 
	idcliente as clientid,
	MAX(limitecreditototal) limitecreditototal,
	MAX(debitoautomatico) debitoautomatico,
	MAX(limitecreditodisponivel) limitecreditodisponivel,
	MAX(diasatraso)	diasatraso
FROM hive.neondw_bi.staging_bi_creditocontratocliente
WHERE datareferencia between date_add('day', -60, current_date) and current_date
AND aceitou = True
AND limitecreditototal > 0
GROUP BY 1
"""

In [64]:
df_limite_credito = pd.read_sql(querie_limite_credito, conn)

In [65]:
wr.s3.to_parquet(df_limite_credito, 's3://neon-datascience-sandbox/segmentacao-cartoes/credito')

{'paths': ['s3://neon-datascience-sandbox/segmentacao-cartoes/credito'],
 'partitions_values': {}}